In [ ]:
# Gọi các thư viện cần thiết 
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
import matplotlib.pyplot as plt  
from keras.layers import Dense, Activation, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import RMSprop # toi uu
from keras.models import Sequential
# Dừng huấn luyện sớm
from keras.callbacks import EarlyStopping
# Để đo lường
from sklearn.metrics import accuracy_score

In [ ]:
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2)    

train_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="../input/genderdataset1/",
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=32,
                                                 directory="../input/genderdataset1/",
                                                 shuffle=True,
                                                 target_size=(150, 150), 
                                                 subset="validation",
                                                 class_mode='categorical')

train_dataset.class_indices

In [ ]:
from keras.layers import Conv2D, MaxPooling2D
model = Sequential()

# Cụm 1
model.add(Conv2D(32,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same',input_shape=(150,150,3)))
model.add(Conv2D(32,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D(2,2))

# Cụm 2
model.add(Conv2D(64,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same')) 
model.add(Conv2D(64,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D(2,2))

# Cụm 3
model.add(Conv2D(128,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same')) 
model.add(Conv2D(128,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D(2,2))

# Cụm 4
model.add(Conv2D(256,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same')) 
model.add(Conv2D(256,(3,3), activation='relu',kernel_initializer='he_uniform',padding='same'))
model.add(MaxPooling2D(2,2))

# Cụm 5
from keras.layers import Dense, Activation, Flatten
model.add(Flatten())
model.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform'))
model.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform'))
model.add(Dense(128, activation = 'relu', kernel_initializer='he_uniform'))

# Cụm 6
model.add(Dense(2))

model.summary()



In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# Huận luyện mô hình 
model.compile(loss='mse',optimizer=RMSprop(),metrics=['accuracy'])
history=model.fit(train_dataset,batch_size=128,epochs=50,validation_data=validation_dataset)

In [ ]:
# Đánh giá độ chính xác của mô hình
score = model.evaluate(train_dataset,verbose=0)
print('Sai số kiểm tra là: ',score[0])
print('Độ chính xác kiểm tra là: ',score[1])

In [ ]:
# Draw plot
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train','Validation'])
plt.show()

In [ ]:
from tensorflow.keras.models import load_model
model.save('GenderV1.h5')

In [ ]:
model_CNN = load_model('GenderV1.h5')
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
filename = "../input/genderdatasetv1/1. Male/face_10.jpg"

predict = ['Male','Female']
predict = np.array(predict)
img = load_img(filename,target_size=(150,150))
plt.imshow(img)

img = img_to_array(img)
img = img.reshape(1,150,150,3)
img = img.astype('float32')
img = img/255


result = np.argmax(model_CNN.predict(img),axis=-1)
predict[result]